# Manipulação de Dados com SQL - DML

Bem-vindo à terceira lição de **SQL**! Agora que você já sabe criar estruturas (DDL) e fazer consultas (SELECT), vamos aprender a **modificar dados existentes**.

## O que é DML?

**DML** (Data Manipulation Language) são os comandos SQL para manipular dados:
- **UPDATE**: Modificar dados existentes
- **DELETE**: Remover dados
- **REPLACE**: Substituir dados

## Nossa Missão 

Vamos trabalhar com situações reais de uma empresa de consultoria e usar o mesmo banco de dados das aulas anteriores:
- **Atualizar informações** que mudaram
- **Remover dados** desnecessários
- **Entender restrições** de integridade referencial
- **Usar cascatas** para atualizações automáticas

In [7]:
# Configuração inicial do banco de dados
import sqlite3
import os
import sys

# Adicionando o diretório de testes ao path
sys.path.append('tests')
from tests_update_delete import *

# Conectando ao banco de dados criado na lição anterior
conn = sqlite3.connect('consultoria.db')

# Habilitando suporte a chaves estrangeiras (muito importante!)
conn.execute("PRAGMA foreign_keys = ON;")

print("✅ Conectado ao banco de dados 'consultoria.db'")
print("✅ Chaves estrangeiras habilitadas")

# Sistema de backup para rollback em caso de erro
backup_conn = None

def backup_db():
    """Cria um backup do banco em memória"""
    global backup_conn
    # Fechar backup anterior se existir
    if backup_conn:
        try:
            backup_conn.close()
        except:
            pass
    backup_conn = sqlite3.connect(':memory:')
    conn.backup(backup_conn)
    print("📁 Backup criado")

def restore_db():
    """Restaura o banco a partir do backup"""
    global backup_conn
    if backup_conn:
        try:
            backup_conn.backup(conn)
            print("🔄 Dados restaurados do backup (ROLLBACK automático)")
        except Exception as e:
            print(f"⚠️ Erro no restore: {e}")
        finally:
            try:
                backup_conn.close()
            except:
                pass
            backup_conn = None
    else:
        print("❌ Nenhum backup disponível")

def validate_and_execute(query, test_function):
    """Valida query em ambiente isolado antes de executar no banco principal"""
    global backup_conn
    
    if not query or not query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    print("🔄 Iniciando transação...")
    # Fazer backup ANTES de qualquer tentativa
    backup_db()

    
    try:
        # Testar a query em um banco temporário isolado
        test_conn = sqlite3.connect(':memory:')
        test_conn.execute("PRAGMA foreign_keys = ON;")
        
        # Copiar estrutura existente para o banco de teste
        for linha in conn.iterdump():
            if not linha.startswith('BEGIN') and not linha.startswith('COMMIT'):
                try:
                    test_conn.execute(linha)
                except:
                    pass  # Ignora erros de estrutura já existente
        
        # Testar a query do usuário no banco isolado
        test_conn.execute(query)
        test_conn.commit()
        
        # Se chegou até aqui, a query é válida sintaticamente
        print("📝 Executando query no banco principal...")
        # Agora executar no banco principal
        conn.execute(query)
        
        # Testar se o resultado está correto
        success = test_function(conn)
        
        if success:
            print("✅ Teste passou! Fazendo COMMIT...")
            conn.commit()
            print("💾 Mudanças salvas permanentemente!")
            # Limpar backup já que mudanças foram confirmadas
            if backup_conn:
                try:
                    backup_conn.close()
                    backup_conn = None
                except:
                    pass
            return True
        else:
            # Se teste falhar, fazer rollback
            print("❌ Teste falhou! Fazendo ROLLBACK...")
            restore_db()
            print("� Banco restaurado ao estado anterior!")
            return False
            
    except Exception as e:
        print(f"❌ Erro na query: {str(e)}")
        print("⚡ Fazendo ROLLBACK automático...")
        restore_db()
        print("💡 Corrija a sintaxe e tente novamente!")
        return False
    finally:
        if 'test_conn' in locals():
            try:
                test_conn.close()
            except:
                pass

print("\n🛠️ Sistema de transações configurado!")
print("\nAgora vamos aprender a modificar dados...")

✅ Conectado ao banco de dados 'consultoria.db'
✅ Chaves estrangeiras habilitadas

🛠️ Sistema de transações configurado!

Agora vamos aprender a modificar dados...


## 📝 Comando UPDATE - Modificando Dados

O comando **UPDATE** permite modificar dados existentes nas tabelas.

### Sintaxe Básica:
```sql
UPDATE nome_da_tabela
SET coluna1 = novo_valor1, 
    coluna2 = novo_valor2
WHERE condicao;
```

### ⚠️ MUITO IMPORTANTE: A Cláusula WHERE
**SEMPRE** use WHERE no UPDATE, senão você modificará **TODAS** as linhas da tabela!

### Exemplos de UPDATE:
```sql
-- Atualizar cidade de um cliente específico
UPDATE clientes 
SET cidade = 'Piracicaba' 
WHERE nome = 'Verde Agro Ltda';

-- Aumentar horas trabalhadas
UPDATE alocacoes 
SET horas_trabalhadas = horas_trabalhadas + 10 
WHERE consultor_id = 1 AND projeto_id = 1;

-- Atualizar múltiplas colunas
UPDATE projetos 
SET data_fim = '2024-12-31', escopo = 'Escopo atualizado'
WHERE id = 3;
```

### Operações Matemáticas no UPDATE:
- `coluna = coluna + 10`
- `coluna = coluna - 5`
- `coluna = coluna * 1.1`
- `coluna = coluna / 2`

## 🔐 Controle de Transações - COMMIT e ROLLBACK

Antes de começarmos a modificar dados, é importante entender que **mudanças precisam ser confirmadas**.

### Comandos Importantes:

#### **COMMIT**
```python
conn.commit()
```
**Confirma** e salva as mudanças permanentemente no banco.

#### **ROLLBACK**
```python
conn.rollback()
```
**Desfaz** as mudanças feitas, voltando ao estado anterior.

### ⚠️ Regra Importante:
- **SEMPRE** use `conn.commit()` após UPDATE, DELETE ou INSERT para salvar as mudanças
- Se algo der errado, use `conn.rollback()` para desfazer

### Na prática:
```python
# Fazer uma mudança
conn.execute("UPDATE clientes SET cidade = 'Nova Cidade' WHERE id = 1")

# OBRIGATÓRIO: Confirmar a mudança
conn.commit()
```


**Nos exercícios não é necessário usar `conn.commit()` ou `conn.rollback()`, pois as mudanças serão aplicadas automaticamente ao final de cada célula.**

In [ ]:
# EXERCÍCIO 1: UPDATE - Cliente mudou de cidade
# A empresa 'Verde Agro Ltda' se mudou de Campinas para Piracicaba
# TODO: Atualize a cidade do cliente 'Verde Agro Ltda' para 'Piracicaba'

query_update_cliente = """

"""

# Quando terminar sua query, execute:
# validate_and_execute(query_update_cliente, test_update_cliente)

# 💡 EXEMPLO DE COMO USAR COMMIT MANUALMENTE:
# conn.execute("UPDATE clientes SET cidade = 'Piracicaba' WHERE nome = 'Verde Agro Ltda'")
# conn.commit()  # ← OBRIGATÓRIO para salvar a mudança!

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: UPDATE cidade do cliente Verde Agro Ltda
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

In [ ]:
# EXERCÍCIO 2: UPDATE - Ajustar horas trabalhadas
# Ana Silva (consultor_id=1) trabalhou mais 10 horas no projeto ERP I (projeto_id=1)
# TODO: Some +10 horas para a alocação onde consultor_id=1 e projeto_id=1

query_update_horas = """

"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_update_horas, test_update_horas)

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: UPDATE horas trabalhadas (+10 para Ana Silva no ERP I)
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

## 🗑️ Comando DELETE - Removendo Dados

O comando **DELETE** remove linhas das tabelas.

### Sintaxe Básica:
```sql
DELETE FROM nome_da_tabela
WHERE condicao;
```

### ⚠️ CUIDADO EXTREMO: A Cláusula WHERE
**SEMPRE** use WHERE no DELETE, senão você apagará **TODOS** os dados da tabela!

### Exemplos de DELETE:
```sql
-- Remover um feedback específico
DELETE FROM feedbacks 
WHERE projeto_id = 5;

-- Remover alocações com poucas horas
DELETE FROM alocacoes 
WHERE horas_trabalhadas < 10;

-- Remover projetos antigos
DELETE FROM projetos 
WHERE data_fim < '2024-01-01';
```

### Restrições de Integridade Referencial:
Lembra das configurações FK que fizemos?

- **ON DELETE RESTRICT**: Impede a exclusão se existem dependências
- **ON DELETE CASCADE**: Remove automaticamente os dados dependentes

### Exemplo de Restrição:
```sql
-- Isso FALHARÁ se o cliente tiver projetos (RESTRICT)
DELETE FROM clientes WHERE id = 1;

-- Isso FUNCIONARÁ e removerá automaticamente alocações e feedbacks (CASCADE)
DELETE FROM projetos WHERE id = 3;
```

In [ ]:
# EXERCÍCIO 3: DELETE - Remover feedback
# O feedback do projeto_id=5 precisa ser removido
# TODO: Delete o feedback onde projeto_id = 5

query_delete_feedback = """

"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_delete_feedback, test_delete_feedback)

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: DELETE feedback do projeto VENDAS V
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

In [ ]:
# EXERCÍCIO 4: DELETE - Testando Restrição ON DELETE RESTRICT
# Vamos tentar deletar um cliente que tem projetos
# TODO: Tente deletar o cliente 'TechCorp Solutions' (id=1)
# EXPECTATIVA: Esta operação deve FALHAR devido à restrição RESTRICT

query_delete_cliente_restrito = """
"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_delete_cliente_restrito, test_delete_cliente_restrito)

🔄 Iniciando transação...
📁 Backup criado
❌ Erro na query: FOREIGN KEY constraint failed
⚡ Fazendo ROLLBACK automático...
🔄 Dados restaurados do backup (ROLLBACK automático)
💡 Corrija a sintaxe e tente novamente!


False

In [ ]:
# EXERCÍCIO 5: DELETE - Testando ON DELETE CASCADE
# Agora vamos deletar o projeto 'LOGISTICA III' (projeto_id=3)
# TODO: Delete o projeto onde id = 3
# EXPECTATIVA: Isso deve remover automaticamente alocações relacionadas (CASCADE)

query_delete_projeto = """

"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_delete_projeto, test_delete_projeto)

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: DELETE projeto LOGISTICA III com CASCADE
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

## 🔄 Comando REPLACE - Substituindo Dados (SQLite)

O comando **REPLACE** combina INSERT + UPDATE.

### Como Funciona:
1. Se a linha **não existir** → age como **INSERT**
2. Se a linha **já existir** (conflito de PK/UNIQUE) → age como **DELETE + INSERT**

### Sintaxe:
```sql
REPLACE INTO nome_da_tabela (coluna1, coluna2, ...)
VALUES (valor1, valor2, ...);
```

### Exemplo de REPLACE:
```sql
-- Se feedback já existe para este projeto, substitui completamente
REPLACE INTO feedbacks (id, projeto_id, nota, comentario)
VALUES (1, 6, 5, 'Comentário completamente novo');

-- Se não existe, insere novo registro
REPLACE INTO feedbacks (projeto_id, nota, comentario)
VALUES (7, 4, 'Novo feedback');
```

### ⚠️ Cuidado com REPLACE:
- **Substitui TODA a linha**, não só as colunas especificadas
- Pode **apagar dados** de colunas não mencionadas
- Para mudanças parciais, prefira **UPDATE**

In [ ]:
# EXERCÍCIO 6: REPLACE - Corrigir nota de feedback
# Use REPLACE para alterar o feedback do projeto_id=6 para nota 5
# TODO: Use REPLACE para alterar a nota do feedback do projeto_id=6 para 5
# Dica: Você precisa incluir TODAS as colunas para não perder dados

query_replace_feedback = """
"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_replace_feedback, test_replace_feedback)

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: REPLACE feedback do projeto FINANCEIRO VI
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

## ⚡ ON UPDATE CASCADE - Atualizações em Cascata

Quando configuramos **ON UPDATE CASCADE** nas FKs, mudanças na chave primária se propagam automaticamente!

### Como Funciona:
```sql
-- Na tabela PROJETOS temos:
FOREIGN KEY (cliente_id) REFERENCES clientes(id) 
ON UPDATE CASCADE

-- Se mudarmos o ID de um cliente:
UPDATE clientes SET id = 40 WHERE id = 4;

-- Automaticamente todos os projetos deste cliente 
-- terão seu cliente_id atualizado para 40!
```

### Casos de Uso:
- **Reestruturação de IDs**: Quando você precisa reorganizar numeração
- **Migração de dados**: Ao mover dados entre ambientes
- **Padronização**: Estabelecer nova convenção de IDs

### ⚠️ Cuidado:
- Use com **muito cuidado** em produção
- Pode afetar **muitas tabelas** simultaneamente
- Sempre faça **backup** antes de usar

In [ ]:
# EXERCÍCIO 7: ON UPDATE CASCADE - Atualizar ID de cliente
# Vamos mudar o ID do cliente_id=4 para 40
# TODO: Atualize o id do cliente onde id=4 para 40
# EXPECTATIVA: Os projetos deste cliente devem ter cliente_id atualizado automaticamente

query_update_cliente_id = """

"""

# Quando terminar sua query, execute a validação:
# validate_and_execute(query_update_cliente_id, test_update_cliente_id)

🔄 Iniciando transação...
📁 Backup criado
📝 Executando query no banco principal...
✅ PASSOU: UPDATE CASCADE cliente ID para 40
✅ Teste passou! Fazendo COMMIT...
💾 Mudanças salvas permanentemente!


True

## 🎉 Parabéns! Você Dominou a Manipulação de Dados!

### O que você aprendeu:

✅ **Controle de Transações**:
- `BEGIN TRANSACTION` para iniciar
- `conn.commit()` para salvar mudanças
- `conn.rollback()` para desfazer mudanças
- Importância da atomicidade de operações

✅ **Comando UPDATE**:
- Modificar dados existentes
- Usar operações matemáticas (+=, -=, *=, /=)
- Importância da cláusula WHERE

✅ **Comando DELETE**:
- Remover dados das tabelas
- Entender restrições de integridade referencial
- Diferença entre RESTRICT e CASCADE

✅ **Comando REPLACE**:
- Substituir dados completamente
- Comportamento híbrido INSERT/UPDATE
- Cuidados ao usar

✅ **Integridade Referencial Avançada**:
- ON DELETE RESTRICT vs CASCADE
- ON UPDATE CASCADE
- Propagação automática de mudanças

### Conceitos Importantes de Segurança:

⚠️ **SEMPRE use WHERE** em UPDATE e DELETE

⚠️ **SEMPRE use `conn.commit()`** para salvar mudanças

⚠️ **SEMPRE use `conn.rollback()`** se algo der errado

⚠️ **Faça backup** antes de operações destrutivas

⚠️ **Teste em ambiente isolado** primeiro

⚠️ **Entenda as cascatas** antes de usar

### Fluxo Seguro de Modificação:
```python
try:
    conn.execute("BEGIN TRANSACTION;")
    # Suas operações SQL aqui
    conn.commit()  # Salva se tudo deu certo
except Exception as e:
    conn.rollback()  # Desfaz se algo falhou
    print(f"Erro: {e}")
```

### Próximos passos:
Agora você sabe criar, consultar e modificar dados com segurança! Na próxima lição, aprenderemos consultas avançadas com JOINs complexos e agregações!

In [17]:
# Verificação final - resumo das modificações feitas
import pandas as pd

print("📊 RESUMO DAS MODIFICAÇÕES REALIZADAS")
print("=" * 50)

try:
    # Reconectar se necessário
    if 'conn' not in locals() or not conn:
        conn = sqlite3.connect('consultoria.db')
    
    # Verificar estado atual das tabelas
    cursor = conn.cursor()
    
    print("\n🔍 Estado atual dos dados:")
    
    # Verificar clientes (deve mostrar Verde Agro em Piracicaba, EduCare com id=40)
    print("\n👥 CLIENTES:")
    cursor.execute("SELECT id, nome, cidade FROM clientes ORDER BY id")
    for linha in cursor.fetchall():
        print(f"   • ID {linha[0]}: {linha[1]} - {linha[2]}")
    
    # Verificar projetos (deve mostrar cliente_id=40 para EDUCACAO IV)
    print("\n📋 PROJETOS:")
    cursor.execute("SELECT id, titulo, cliente_id FROM projetos ORDER BY id")
    for linha in cursor.fetchall():
        print(f"   • ID {linha[0]}: {linha[1]} (Cliente: {linha[2]})")
    
    # Verificar feedbacks (deve mostrar projeto FINANCEIRO VI com nota 5)
    print("\n⭐ FEEDBACKS:")
    cursor.execute("""
        SELECT f.id, p.titulo, f.nota 
        FROM feedbacks f 
        JOIN projetos p ON f.projeto_id = p.id 
        ORDER BY f.id
    """)
    for linha in cursor.fetchall():
        print(f"   • ID {linha[0]}: {linha[1]} - Nota: {linha[2]}")
    
    # Contar registros para ver se CASCADE funcionou
    print("\n📊 Contagem de registros:")
    cursor.execute("SELECT COUNT(*) FROM alocacoes")
    alocacoes_count = cursor.fetchone()[0]
    print(f"   • Alocações: {alocacoes_count} registros")
    
    cursor.execute("SELECT COUNT(*) FROM feedbacks")
    feedbacks_count = cursor.fetchone()[0]
    print(f"   • Feedbacks: {feedbacks_count} registros")
    
    print(f"\n✅ Exercícios DML concluídos com sucesso!")
    print(f"🔄 Integridade referencial funcionando corretamente!")
    
except Exception as e:
    print(f"❌ Erro ao verificar dados: {str(e)}")
    print("💡 Execute os exercícios primeiro!")

finally:
    if 'conn' in locals():
        conn.close()
    print(f"\n🔌 Conexão fechada. Lição 3 concluída!")

📊 RESUMO DAS MODIFICAÇÕES REALIZADAS

🔍 Estado atual dos dados:

👥 CLIENTES:
   • ID 1: TechCorp Solutions - São Paulo
   • ID 2: Verde Agro Ltda - Piracicaba
   • ID 3: MetalMax Indústrias - Santos
   • ID 5: FastLogistic S.A. - São José dos Campos
   • ID 40: EduCare Ensino - Ribeirão Preto

📋 PROJETOS:
   • ID 1: ERP I (Cliente: 1)
   • ID 2: MARKETING II (Cliente: 2)
   • ID 4: EDUCACAO IV (Cliente: 40)
   • ID 5: VENDAS V (Cliente: 3)
   • ID 6: FINANCEIRO VI (Cliente: 1)

⭐ FEEDBACKS:
   • ID 1: ERP I - Nota: 5
   • ID 2: MARKETING II - Nota: 4
   • ID 3: EDUCACAO IV - Nota: 5
   • ID 5: FINANCEIRO VI - Nota: 5

📊 Contagem de registros:
   • Alocações: 8 registros
   • Feedbacks: 4 registros

✅ Exercícios DML concluídos com sucesso!
🔄 Integridade referencial funcionando corretamente!

🔌 Conexão fechada. Lição 3 concluída!
